In [6]:
import pandas as pd

URL = "https://docs.google.com/spreadsheets/d/1YwuNz9lKEx8zj3th5hHfI1Z7i2WKUGexfqPnrxn6jiw/export?format=csv"

RENAME_MAP = {
    "Horodateur": "Timestamp",
    "Pour vous, être productif, c'est avant tout...": "Definition_productivite",
    "En moyenne, combien d'heures dormez-vous par nuit?": "Sommeil_moyen",
    "Combien d'heures avez-vous dormi la nuit dernière?": "Sommeil_nuit_derniere",
    "À quelle fréquence avez-vous un sommeil réparateur ?": "Sommeil_reparateur",
    "Comment évalueriez-vous votre hygiène de vie actuelle ?": "Hygiene_vie",
    "À quelle fréquence pratiquez-vous une activité physique (sport, marche active, etc.) ?": "Frequence_sport",
    "En moyenne, combien de litres d'eau bois-tu par jour ?": "Eau_litres",
    "Consommation quotidienne de café ou boissons énergétiques (nombre de tasses/verres)": "Cafe",
    "Avez-vous l'impression d'avoir été efficace dans vos tâches aujourd'hui ?": "Efficacite_aujourdhui",
    "Quel est votre niveau de stress général ces derniers jours ?": "Stress",
    "Niveau moyen de productivité ces 7 derniers jours.": "Productivite_7j",
    "Niveau d'énergie aujourd'hui.": "Energie",
    "Quel est votre âge ?": "Age",
    "Quelle est votre situation actuelle ?": "Situation",
}

df = pd.read_csv(URL)

df.columns = (
    df.columns
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .str.replace("\u2019", "'", regex=False)  # ✅ apostrophe typographique → droite
    .str.replace("\u2018", "'", regex=False)  # ✅ apostrophe ouvrante → droite
)

print("=== COLONNES AVANT RENOMMAGE ===")
for col in df.columns.tolist():
    print(repr(col))

df = df.rename(columns=RENAME_MAP)

print("\n=== COLONNES APRÈS RENOMMAGE ===")
for col in df.columns.tolist():
    print(repr(col))

print("\n=== COLONNES NON RENOMMÉES (encore en français) ===")
colonnes_cibles = set(RENAME_MAP.values())
for col in df.columns.tolist():
    if col not in colonnes_cibles and col != "Timestamp":
        print(repr(col))

=== COLONNES AVANT RENOMMAGE ===
'Horodateur'
"Pour vous, être productif, c'est avant tout..."
"En moyenne, combien d'heures dormez-vous par nuit?"
"Combien d'heures avez-vous dormi la nuit dernière?"
'À quelle fréquence avez-vous un sommeil réparateur ?'
'Comment évalueriez-vous votre hygiène de vie actuelle ?'
'À quelle fréquence pratiquez-vous une activité physique (sport, marche active, etc.) ?'
"En moyenne, combien de litres d'eau bois-tu par jour ?"
'Consommation quotidienne de café ou boissons énergétiques (nombre de tasses/verres)'
"Avez-vous l'impression d'avoir été efficace dans vos tâches aujourd'hui ?"
'Quel est votre niveau de stress général ces derniers jours ?'
'Niveau moyen de productivité ces 7 derniers jours.'
"Niveau d'énergie aujourd'hui."
'Quel est votre âge ?'
'Quelle est votre situation actuelle ?'

=== COLONNES APRÈS RENOMMAGE ===
'Timestamp'
'Definition_productivite'
'Sommeil_moyen'
'Sommeil_nuit_derniere'
'Sommeil_reparateur'
'Hygiene_vie'
'Frequence_sport'
'E

In [ ]:
d

,Horodateur,"Pour vous, être productif, c'est avant tout...","En moyenne, combien d'heures dormez-vous par nuit?",Combien d'heures avez-vous dormi la nuit dernière?,À quelle fréquence avez-vous un sommeil réparateur ?,Comment évalueriez-vous votre hygiène de vie actuelle ?,"À quelle fréquence pratiquez-vous une activité physique (sport, marche active, etc.) ?","En moyenne, combien de litres d'eau bois-tu par jour ?",Consommation quotidienne de café ou boissons énergétiques (nombre de tasses/verres),Avez-vous l'impression d'avoir été efficace dans vos tâches aujourd'hui ?,Quel est votre niveau de stress général ces derniers jours ?,Niveau moyen de productivité ces 7 derniers jours.,Niveau d’énergie aujourd’hui.,Quel est votre âge ?,Quelle est votre situation actuelle ?
0,25/02/2026 12:28:28,Travailler vite et efficacement,06:00,05:00,Parfois (1 à 2 fois par semaine),Moyenne,3 à 4 fois par semaine,2,1.000000e+00,"Mou du genou (Petite forme, beaucoup de distra...",3,3,4,18-25 ans,Étudiant(e)
1,25/02/2026 12:29:20,Accomplir toutes ses tâches de la journée,06:00,05:00,Parfois (1 à 2 fois par semaine),Très bonne,1 à 2 fois par semaine,3,1.000000e+00,"Propre (Efficacité correcte, boulot fait)",3,2,3,18-25 ans,Étudiant(e)
2,25/02/2026 12:34:21,Travailler vite et efficacement,07:00,06:00,Souvent (3 à 5 fois par semaine),Très bonne,Tous les jours,2,0.000000e+00,"Propre (Efficacité correcte, boulot fait)",3,3,3,18-25 ans,Étudiant(e)
3,25/02/2026 12:35:16,Avoir l'esprit calme et concentré,05:00,06:00,Parfois (1 à 2 fois par semaine),Moyenne,1 à 2 fois par semaine,3,1.000000e+00,"Propre (Efficacité correcte, boulot fait)",4,3,3,18-25 ans,Étudiant(e)
4,25/02/2026 12:39:39,Travailler vite et efficacement,06:00,05:00,Souvent (3 à 5 fois par semaine),Moyenne,Jamais,3,0.000000e+00,"Mou du genou (Petite forme, beaucoup de distra...",2,3,3,18-25 ans,Travailleur / Travailleuse
5,25/02/2026 12:42:39,Accomplir toutes ses tâches de la journée,06:00,05:00,Parfois (1 à 2 fois par semaine),Très bonne,1 à 2 fois par semaine,2,0.000000e+00,"Mou du genou (Petite forme, beaucoup de distra...",4,2,2,18-25 ans,Étudiant(e)
6,25/02/2026 12:43:06,Travailler vite et efficacement,17:00,08:00,Parfois (1 à 2 fois par semaine),Pas terrible,1 à 2 fois par semaine,1,0.000000e+00,"Propre (Efficacité correcte, boulot fait)",1,3,3,18-25 ans,Étudiant(e)
7,25/02/2026 12:51:32,Travailler vite et efficacement,"5,5",5,Jamais,Très bonne,3 à 4 fois par semaine,1,1.000000e+00,"Propre (Efficacité correcte, boulot fait)",1,3,4,18-25 ans,Étudiant(e)
8,25/02/2026 13:09:21,Travailler vite et efficacement,5 à 6,6,Parfois (1 à 2 fois par semaine),Moyenne,Tous les jours,1,2.000000e+00,"Propre (Efficacité correcte, boulot fait)",2,3,4,18-25 ans,Les deux
9,25/02/2026 13:09:57,Avoir l'esprit calme et concentré,5- 6h,5 h,Jamais,Moyenne,3 à 4 fois par semaine,2,1.000000e+00,"Mou du genou (Petite forme, beaucoup de distra...",4,2,2,18-25 ans,Travailleur / Travailleuse
